In [21]:
import pandas as pd
import numpy as np


In [22]:
df=pd.read_csv('sushi.csv')
df.head()

,Name,Rating,Reviews,Price,Location
0,Yama Sushi Sake Attitude,5.0,214,$$,"{'address1': '11709 National Blvd', 'address2'..."
1,Uzumaki,4.5,669,$$,"{'address1': '11060 Washington Blvd', 'address..."
2,Izakaya Osen - Los Angeles,4.5,1631,$$,"{'address1': '2903 Sunset Blvd', 'address2': '..."
3,Roll Call - Koreatown,4.5,532,$$,"{'address1': '3183 Wilshire Blvd', 'address2':..."
4,K-ZO,4.0,923,$$$,"{'address1': '9240 Culver Blvd', 'address2': N..."


In [20]:
len(df)

1000

In [ ]:
# Assuming your DataFrame is called 'df' and the column is 'location'
df['Number of Locations'] = df['Location'].apply(lambda x: len(x.keys()))
df['Price'] = df['Price'].str.count('$')


# Define a function to extract the street name
def extract_street_name(address):
    # Split the address by whitespace
    parts = address.split()
    
    # Remove the last word (e.g., 'Blvd', 'Ave', etc.)
    street_parts = parts[:-1]
    
    # Join the remaining parts to form the street name
    street_name = ' '.join(street_parts)
    
    return street_name

# Apply the function to the 'address' column to create a new 'street_name' column
df['street_name'] = df['address'].apply(extract_street_name)

# Print the updated DataFrame
print(df)
